In [72]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint as pp
import csv
import datetime
from dateutil.parser import parse
from pandas.tools.plotting import scatter_matrix
from patsy import dmatrices
import statsmodels.formula.api as smf
from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm
import seaborn as sns
import statsmodels.api as sm
import matplotlib as mpl
from numpy.random import randn
from scipy import stats
from pprint import pprint as pp
from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import roc_curve, auc
from sklearn.naive_bayes import GaussianNB
from sklearn.learning_curve import learning_curve
from sklearn.cross_validation import cross_val_score
import urllib2
from pandas import Series
from pandas import merge, concat
%matplotlib inline

sns.set_palette("deep", desat=.8)
sns.set_context(rc={"figure.figsize": (14, 8)})

In [2]:
url           = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/cleveland.data"

newrl         = urllib2.urlopen(url)
newdata_df    = pd.read_csv(newrl, header=-1)

newdata_df.head()

,0
0,1 0 63 1 -9 -9 -9
1,-9 1 145 1 233 -9 50 20
2,1 -9 1 2 2 3 81 0
3,0 0 0 0 1 10.5 6 13
4,150 60 190 90 145 85 0 0


In [9]:
cleve_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/cleveland.data"
newrl = urllib2.urlopen(cleve_url)
colus = list(range(1,15))
cleve_df = pd.read_csv("switz.data", header=None, delimiter="name\n")
# id_data = cleve_df[0].str.strip("[]").str.split(",").tolist()
# df_id = pd.DataFrame(id_data, columns=colus, index=cleve_df.index, dtype=int)
# cleve_df = cleve_df.join(df_id)
cleve_df.columns = ["name"]
r1 = cleve_df[0::10]
r2 = cleve_df[1::10]
r3 = cleve_df[2::10]
r4 = cleve_df[3::10]
r5 = cleve_df[4::10]
r6 = cleve_df[5::10]
r7 = cleve_df[6::10]
r8 = cleve_df[7::10]
r9 = cleve_df[8::10]
r10 = cleve_df[9::10]
# cleve_df

In [4]:
cleve_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/cleveland.data"
newrl = urllib2.urlopen(cleve_url)
cleve_df = pd.read_csv(newrl, header=None)
# cleve_df = read_table(newrl, index_col=[0,1,2])
cleve_df.head()

,0
0,1 0 63 1 -9 -9 -9
1,-9 1 145 1 233 -9 50 20
2,1 -9 1 2 2 3 81 0
3,0 0 0 0 1 10.5 6 13
4,150 60 190 90 145 85 0 0


In [94]:
tdf = pd.read_csv("data.csv")
tdf = tdf.replace(-9, np.nan)
tdf = tdf.fillna(tdf.mean())
tdf["num"] = tdf["num"].replace(2,1).replace(3,1).replace(4,1)
drops = ["id", "ccf", "lvx1", "pncaden", "exeref","earlobe", "exerckm", "restckm", "restef", "lvx2", "lvx3", "lvx4", "lvf", "cathef", "junk","ekgmo", "ekgday", "proto", "dig", "prop", "nitr", "pro", "diuretic", "thalsev", "cmo", "cday", "lmt", "ladprox", "laddist", "diag","cxmain", "ramus", "om1", "om2", "rcaprox", "rcadist"]
tdf = tdf.drop(drops, axis=1)
# tdf.head()
tdf.describe()

,age,sex,painloc,painexer,relrest,cp,trestbps,htn,chol,smoke,...,slope,rldv5,rldv5e,ca,restwm,exerwm,thal,thalpul,cyr,num
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,...,699.000000,699.000000,699.000000,699.000000,699,699,699.000000,699.000000,699.000000,699.000000
mean,51.801144,0.739628,0.918465,0.553957,0.597122,3.181688,131.757184,0.456587,204.473373,0.571429,...,1.697154,13.758112,63.405145,0.702422,0,0,4.944882,0.302326,83.918841,0.494993
std,9.189922,0.439152,0.211516,0.384208,0.379104,0.951492,18.577722,0.487290,108.782481,0.110815,...,0.487488,3.849639,59.766186,0.677523,0,0,1.431253,0.113991,1.665190,0.500333
min,28.000000,0.000000,0.000000,0.000000,0.000000,1.000000,80.000000,0.000000,0.000000,0.000000,...,1.000000,3.000000,2.000000,0.000000,0,0,1.000000,0.000000,81.000000,0.000000
25%,45.000000,0.000000,0.918465,0.000000,0.597122,2.000000,120.000000,0.000000,184.000000,0.571429,...,1.000000,13.000000,13.000000,0.702422,0,0,3.000000,0.302326,83.000000,0.000000
50%,52.000000,1.000000,1.000000,0.553957,0.597122,4.000000,130.000000,0.000000,226.000000,0.571429,...,1.697154,13.758112,45.000000,0.702422,0,0,4.944882,0.302326,84.000000,0.000000
75%,58.000000,1.000000,1.000000,1.000000,1.000000,4.000000,140.000000,1.000000,269.000000,0.571429,...,2.000000,13.758112,108.000000,0.702422,0,0,6.000000,0.302326,85.000000,1.000000
max,77.000000,1.000000,1.000000,1.000000,1.000000,4.000000,200.000000,1.000000,603.000000,1.000000,...,3.000000,31.000000,270.000000,9.000000,0,0,7.000000,1.000000,87.000000,1.000000


In [95]:
cols = tdf.columns.tolist()
cols = cols[-1:] + cols[:-1]
tdf = tdf[cols]
x_all = tdf.iloc[:,1:]
y_all = tdf.iloc[:,0]
tdf.head()

,num,age,sex,painloc,painexer,relrest,cp,trestbps,htn,chol,...,oldpeak,slope,rldv5,rldv5e,ca,restwm,exerwm,thal,thalpul,cyr
0,0,40,1,1,0,0,2,140,0,289.000000,...,0,1.697154,26,20,0.702422,0,0,4.944882,0.302326,84
1,1,49,0,1,0,0,3,160,1,180.000000,...,1,2.000000,14,13,0.702422,0,0,4.944882,0.302326,84
2,0,37,1,1,0,0,2,130,0,283.000000,...,0,1.697154,17,14,0.702422,0,0,4.944882,0.302326,84
3,1,48,0,1,1,1,4,138,0,214.000000,...,15,2.000000,19,22,0.702422,0,0,4.944882,0.302326,84
4,0,54,1,1,0,1,3,150,0,204.473373,...,0,1.697154,13,9,0.702422,0,0,4.944882,0.302326,84


In [96]:
train, test = train_test_split(tdf, train_size = 0.8)
x_train  = train.iloc[:,1:]
y_train  = train.iloc[:,0]
x_test   = test.iloc[:,1:]
y_test   = test.iloc[:,0]

In [97]:
model = LogisticRegression()
avg_acc = np.mean(cross_val_score(model, x_all, y_all, scoring='recall'))
avg_precision = np.mean(cross_val_score(model, x_all, y_all, scoring='precision'))
avg_recall = np.mean(cross_val_score(model, x_all, y_all, scoring='recall'))
avg_f1 = np.mean(cross_val_score(model, x_all, y_all, scoring='f1'))
print "Accuracy: %f" % avg_acc
print "Precision: %f" % avg_precision
print "Recall: %f" % avg_recall
print "F1: %f" % avg_f1

[0.61069465267366319,
 0.85842449328208659,
 0.61069465267366319,
 0.6711551793253111]

In [78]:
knn_model = KNeighborsClassifier(n_neighbors=16)
knn_model.fit(x_train, y_train)
y_pred  = knn_model.predict(x_test)
a_score = accuracy_score(y_test, y_pred)
print "Accuracy is:"
a_score

Accuracy is:


0.58571428571428574